In [1]:
import subprocess
import os
import time
import numpy as np
import open3d as o3d
import re

from open3d.examples.visualization.to_mitsuba import dataset

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [86]:
dataset = 'Dancer'
qp = 10
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

In [87]:
original_input_dir = os.path.join(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data', os.path.join(dataset, 'meshes'))
decoded_input_dir = os.path.join(r'G:\PycharmProjects\Mesh_Editing\Results\decode_Draco', dataset)
files = os.listdir(original_input_dir)
original_obj_files = [file for file in files if file.endswith('.obj')]

original_obj_files.sort(key=extract_number)
decoded_obj_path = os.path.join(decoded_input_dir, f'{dataset}_qp_{qp}')
decoded_files = os.listdir(decoded_obj_path)
decoded_obj_files = [file for file in decoded_files if file.endswith('.obj')]
test_original_mesh = o3d.io.read_triangle_mesh(os.path.join(original_input_dir, original_obj_files[0]))
test_decoded_mesh = o3d.io.read_triangle_mesh(os.path.join(decoded_obj_path, decoded_obj_files[0]))
print(test_original_mesh, test_decoded_mesh)

TriangleMesh with 19724 points and 39380 triangles. TriangleMesh with 19445 points and 39372 triangles.


## Point-to-point error (D1), D1-PSNR (peak signal-to-noise ratio)

In [60]:
def compute_D1_psnr(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    
    return psnr

In [55]:
A = (1, 2, 3)
print(np.linalg.norm(A))
print(np.mean(A))

3.7416573867739413
2.0


In [61]:
d1 = max(compute_D1_psnr(test_original_mesh, test_decoded_mesh), compute_D1_psnr(test_decoded_mesh, test_original_mesh))
print(d1)

97.86869073879592


## Point-to-plane error (D2), D2-PSNR (peak signal-to-noise ratio)

In [75]:
def compute_D2_psnr(original_mesh, decoded_mesh):
    decoded_mesh.compute_vertex_normals()
    
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_decoded.normals = o3d.utility.Vector3dVector(decoded_mesh.vertex_normals)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.dot((original_vertices[i] - decoded_vertices[index])[0], np.array(pcd_decoded.normals)[index][0]))
    MSE = MSE / len(original_vertices)
    
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    
    return psnr

In [76]:
d2 = max(compute_D2_psnr(test_original_mesh, test_decoded_mesh), compute_D2_psnr(test_decoded_mesh, test_original_mesh))
print(d2)

102.6052241255284


## RMSE

In [81]:
def compute_MSE_RMSE(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    RMSE =np.sqrt(MSE)
    
    return np.log10(MSE), np.log10(RMSE)

In [83]:
mse, rmse = compute_MSE_RMSE(test_original_mesh, test_decoded_mesh)
print(mse, rmse)

-9.120304100036503 -4.560152050018251


## Hausdorff distance

In [92]:
from scipy.spatial.distance import directed_hausdorff
test_original_vertices = np.array(test_original_mesh.vertices)
test_decoded_vertices = np.array(test_decoded_mesh.vertices)
hausdorff = directed_hausdorff(test_original_vertices, test_decoded_vertices)
print(hausdorff[0] * 1e4)

14.685178498337223
